In [49]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import wandb
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
from torchkan import KAN
from KACnet import KAC_Net
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import torch
wandb.init(project="kan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: quanbo-li (two-phase). Use `wandb login --relogin` to force relogin


In [2]:
def train_and_validate_model(model, epochs, learning_rate, train_loader, val_loader, model_name):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            predicted_y = model(x)
            loss = loss_fn(predicted_y, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        scheduler.step()
        avg_loss = total_loss / len(train_loader)
        #wandb.log({f"{model_name} Train Loss": avg_loss})

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                predicted_y = model(x)
                val_loss = loss_fn(predicted_y, y.unsqueeze(1))
                total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(val_loader)
        #wandb.log({f"{model_name} Validation Loss": avg_val_loss})
        print(f"Epoch {epoch}, {model_name} Train Loss: {avg_loss}, Validation Loss: {avg_val_loss}")
# Evaluation function
def evaluate_model(model, eval_loader, model_name):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for x, y in eval_loader:
            predicted_y = model(x)
            predictions.extend(predicted_y.squeeze().cpu().numpy())
            actuals.extend(y.cpu().numpy())
    return predictions, actuals



In [25]:
# Simple MLP model matching KAN structure
class MLP(nn.Module):
    def __init__(self, layers):
        super(MLP, self).__init__()
        mlp_layers = []
        for i in range(len(layers) - 1):
            mlp_layers.append(nn.Linear(layers[i], layers[i+1]))
            if i < len(layers) - 2:
                mlp_layers.append(nn.ReLU())
        self.model = nn.Sequential(*mlp_layers)

    def forward(self, x):
        return self.model(x)



In [13]:
layers=[4, 64, 64, 32, 32, 16, 2]
kan_model = KAN(layers)
display(kan_model)

KAN(
  (base_activation): GELU(approximate='none')
  (base_weights): ParameterList(
      (0): Parameter containing: [torch.float32 of size 64x4]
      (1): Parameter containing: [torch.float32 of size 64x64]
      (2): Parameter containing: [torch.float32 of size 32x64]
      (3): Parameter containing: [torch.float32 of size 32x32]
      (4): Parameter containing: [torch.float32 of size 16x32]
      (5): Parameter containing: [torch.float32 of size 2x16]
  )
  (spline_weights): ParameterList(
      (0): Parameter containing: [torch.float32 of size 64x4x8]
      (1): Parameter containing: [torch.float32 of size 64x64x8]
      (2): Parameter containing: [torch.float32 of size 32x64x8]
      (3): Parameter containing: [torch.float32 of size 32x32x8]
      (4): Parameter containing: [torch.float32 of size 16x32x8]
      (5): Parameter containing: [torch.float32 of size 2x16x8]
  )
  (layer_norms): ModuleList(
    (0-1): 2 x LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2-3): 2

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取xlsx文件
file_path = 'D:\OneDrive - Officials\OneDrive - Mraz Cindy\done\毕设资料\计算公式说明\数据库.xlsx'  # 替换为你的xlsx文件路径
df = pd.read_excel(file_path, engine='openpyxl')

# 打乱行顺序并按8:2的比例分成训练集和测试集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# 提取第2，3，4，5列的数据
input = train_df.iloc[:, 1:5]
output = train_df.iloc[:, 5:7]
test_input = test_df.iloc[:, 1:5]
test_label = test_df.iloc[:, 5:7]
# 将DataFrame转换为numpy数组并调整其维度为4
array1 = input.to_numpy()
array2 = output.to_numpy()
array3 = test_input.to_numpy()
array4 = test_label.to_numpy()
input=torch.tensor(array1, dtype=torch.float32)
ouput=torch.tensor(array2, dtype=torch.float32)
test_input=torch.tensor(array3, dtype=torch.float32)
test_label=torch.tensor(array4, dtype=torch.float32)
def normalize_columns(tensor):
    # 确保输入是2D张量
    assert tensor.dim() == 2, "Input tensor must be 2D"
    
    # 获取最小值和最大值
    col_min = tensor.min(dim=0, keepdim=True).values
    col_max = tensor.max(dim=0, keepdim=True).values
    
    # 防止除以零的情况
    denom = col_max - col_min
    denom[denom == 0] = 1  # 如果列中所有值相等，避免除以零
    
    # 进行归一化
    normalized_tensor = (tensor - col_min) / denom
    return normalized_tensor
# 对每一列进行归一化
input= normalize_columns(input)
ouput= normalize_columns(ouput)
test_input= normalize_columns(test_input)
test_label= normalize_columns(test_label)
dataset={'train_input':input,'test_input':test_input,'train_label':ouput,'test_label':test_label}

# 输出结果
print("训练集样本数:", len(train_df))
print("测试集样本数:", len(test_df))

训练集样本数: 307
测试集样本数: 77


In [26]:
layers1=[4, 64, 64, 32, 32, 16, 1]
KAC_Net=KAC_Net(layers1)
display(KAC_Net)

KAC_Net(
  (base_activation): SiLU()
  (base_weights): ParameterList(
      (0): Parameter containing: [torch.float32 of size 64x4]
      (1): Parameter containing: [torch.float32 of size 64x64]
      (2): Parameter containing: [torch.float32 of size 32x64]
      (3): Parameter containing: [torch.float32 of size 32x32]
      (4): Parameter containing: [torch.float32 of size 16x32]
      (5): Parameter containing: [torch.float32 of size 1x16]
  )
  (poly_weights): ParameterList(
      (0): Parameter containing: [torch.float32 of size 64x16]
      (1): Parameter containing: [torch.float32 of size 64x256]
      (2): Parameter containing: [torch.float32 of size 32x256]
      (3): Parameter containing: [torch.float32 of size 32x128]
      (4): Parameter containing: [torch.float32 of size 16x128]
      (5): Parameter containing: [torch.float32 of size 1x64]
  )
  (layer_norms): ModuleList(
    (0-1): 2 x LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2-3): 2 x LayerNorm((32,), eps

In [47]:
print(ouput
      )

tensor([[0.7454, 0.2318],
        [0.4363, 0.4717],
        [0.3810, 0.2224],
        [0.4316, 0.0911],
        [0.4478, 0.2405],
        [0.6133, 0.6856],
        [0.3451, 0.1812],
        [0.4035, 0.2383],
        [0.3175, 0.7977],
        [0.6342, 0.9059],
        [0.3399, 0.1814],
        [0.7746, 0.1186],
        [0.2827, 0.4222],
        [0.3195, 0.3665],
        [0.2583, 0.0472],
        [0.7440, 0.1156],
        [0.3376, 0.4244],
        [0.2591, 0.0464],
        [0.2664, 0.0479],
        [0.2768, 0.0957],
        [0.2962, 0.5465],
        [0.4832, 0.5239],
        [0.2100, 0.0648],
        [0.3685, 0.1695],
        [0.3592, 0.2739],
        [0.2897, 0.7471],
        [0.9985, 0.5095],
        [0.6502, 0.1128],
        [0.3115, 0.0796],
        [0.0000, 0.5281],
        [0.8494, 0.2458],
        [0.8640, 0.6344],
        [0.2770, 0.1343],
        [0.2988, 0.1089],
        [0.1976, 0.0390],
        [0.3084, 0.1427],
        [0.7358, 0.0826],
        [0.2362, 0.0954],
        [0.5

In [51]:
wandb.init(project="kan")
dimension=4
# Define model layers
layers = [dimension, 64, 64, 32, 32, 16, 2]
x_data=torch.cat((input,test_input),0)
y_data=torch.cat((ouput,test_label),0)

# 定义一个TensorDataset对象，将x_data和y_data传入
dataset = TensorDataset(x_data, y_data)
# 计算训练集和验证集的数量
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
# 将数据集划分为训练集和验证集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# 创建训练集的DataLoader对象，批量大小为32，打乱数据
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 创建验证集的DataLoader对象，批量大小为32，不打乱数据
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Initialize and train the KAN model
kan_model = KAN(layers)
train_and_validate_model(kan_model, epochs=50, learning_rate=0.001, train_loader=train_loader, val_loader=val_loader, model_name=f"KAN")

    # Initialize and train the MLP model
mlp_model = MLP(layers)
train_and_validate_model(mlp_model, epochs=50, learning_rate=0.001, train_loader=train_loader, val_loader=val_loader, model_name=f"MLP")

    # Evaluate both models
kan_predictions, kan_actuals = evaluate_model(kan_model, val_loader, f"KAN")
mlp_predictions, mlp_actuals = evaluate_model(mlp_model, val_loader, f"MLP")

    # Log results to wandb
kan_data = [[pred, act] for pred, act in zip(kan_predictions, kan_actuals)]
mlp_data = [[pred, act] for pred, act in zip(mlp_predictions, mlp_actuals)]
wandb.log({
        f"KAN Predictions vs Actuals": wandb.Table(data=kan_data, columns=["KAN Predictions", "Actuals"]),
        f"MLP Predictions vs Actuals": wandb.Table(data=mlp_data, columns=["MLP Predictions", "Actuals"])
    })

    # Save model states
torch.save(kan_model.state_dict(), f"kan inverse.pth")
torch.save(mlp_model.state_dict(), f"mlp inverse.pth")
wandb.save(f"kan inverse.pth")
wandb.save(f"mlp inverse.pth")


Epoch 0, KAN Train Loss: 0.42429396841261124, Validation Loss: 0.36561378836631775
Epoch 1, KAN Train Loss: 0.37964848677317303, Validation Loss: 0.35841965675354004
Epoch 2, KAN Train Loss: 0.34265784753693473, Validation Loss: 0.3250409960746765
Epoch 3, KAN Train Loss: 0.31973867615063983, Validation Loss: 0.3107745796442032
Epoch 4, KAN Train Loss: 0.3024112814002567, Validation Loss: 0.2973761707544327
Epoch 5, KAN Train Loss: 0.28700319760375553, Validation Loss: 0.27531349658966064
Epoch 6, KAN Train Loss: 0.26408912738164264, Validation Loss: 0.25749843567609787
Epoch 7, KAN Train Loss: 0.24823783006933, Validation Loss: 0.2453388012945652
Epoch 8, KAN Train Loss: 0.23966077963511148, Validation Loss: 0.2339014895260334
Epoch 9, KAN Train Loss: 0.22208855715062883, Validation Loss: 0.22336682677268982
Epoch 10, KAN Train Loss: 0.22161036729812622, Validation Loss: 0.21842137724161148
Epoch 11, KAN Train Loss: 0.215628981590271, Validation Loss: 0.21358750760555267
Epoch 12, KAN

OSError: [WinError 1314] 客户端没有所需的特权。: 'f:\\mycode\\pykan\\torchkan-main\\kan inverse.pth' -> 'f:\\mycode\\pykan\\torchkan-main\\wandb\\run-20240622_190153-zvv7w4pa\\files\\kan inverse.pth'